In [0]:
from keras.applications import VGG16 , ResNet50 , VGG19
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 3s 0us/step


In [0]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f92f00f0208> False
<keras.layers.convolutional.Conv2D object at 0x7f92a899e320> False
<keras.layers.convolutional.Conv2D object at 0x7f92a899e4a8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f92a8944ba8> False
<keras.layers.convolutional.Conv2D object at 0x7f92a8944630> False
<keras.layers.convolutional.Conv2D object at 0x7f92a80f9940> False
<keras.layers.pooling.MaxPooling2D object at 0x7f92a810fcc0> False
<keras.layers.convolutional.Conv2D object at 0x7f92a810fac8> False
<keras.layers.convolutional.Conv2D object at 0x7f92a80bccc0> False
<keras.layers.convolutional.Conv2D object at 0x7f92a80d4e48> False
<keras.layers.pooling.MaxPooling2D object at 0x7f92a807c5f8> False
<keras.layers.convolutional.Conv2D object at 0x7f92a807cb38> False
<keras.layers.convolutional.Conv2D object at 0x7f92a80287b8> False
<keras.layers.convolutional.Conv2D object at 0x7f92a8043f60> False
<keras.layers.pooling.MaxPooling2D object at 0x7f92a7fec7f0>

In [0]:
from keras import models
from keras import layers
from keras import optimizers
from keras import Sequential 
from keras.layers import BatchNormalization , Conv2D 
# Create the model
model = Sequential()

model.add(Conv2D(10, kernel_size = (1,1), input_shape=(64,64,3),padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(3, kernel_size = (1,1), padding = 'same', activation = 'relu'))
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))
 
# # Show a summary of the model. Check the number of trainable parameters
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 10)        40        
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 10)        40        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 3)         33        
_________________________________________________________________
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
___________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10

 
train_generator = train_datagen.flow_from_directory(
        "./Dataset/training_set/",
        target_size=(64, 64),
        batch_size=train_batchsize,
        class_mode='categorical')
 


Found 40001 images belonging to 5 classes.


In [0]:
validation_generator = validation_datagen.flow_from_directory(
        "./Dataset/test_set",
        target_size=(64, 64),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 9554 images belonging to 5 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [0]:

model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      )

Epoch 1/5
401/400 [==============================] - 115s 286ms/step - loss: 0.3220 - acc: 0.8822 - val_loss: 0.2064 - val_acc: 0.9299
Epoch 2/5
401/400 [==============================] - 111s 276ms/step - loss: 0.1330 - acc: 0.9556 - val_loss: 0.3983 - val_acc: 0.8862
Epoch 3/5
401/400 [==============================] - 110s 275ms/step - loss: 0.1057 - acc: 0.9653 - val_loss: 0.1787 - val_acc: 0.9344
Epoch 4/5
401/400 [==============================] - 110s 275ms/step - loss: 0.0927 - acc: 0.9706 - val_loss: 0.1954 - val_acc: 0.9402
Epoch 5/5
401/400 [==============================] - 111s 276ms/step - loss: 0.0905 - acc: 0.9728 - val_loss: 0.6695 - val_acc: 0.8770


In [0]:
scores = model.evaluate_generator(validation_generator,steps=validation_generator.samples/validation_generator.batch_size)
print("accuracy is ", scores[1])

accuracy is  0.877014862432349


In [0]:
model.save_weights("weights.hdf5",overwrite=True)

In [0]:
model_json = model.to_json()
with open("model.json", "w") as model_file:
    model_file.write(model_json)
print("Model has been saved.")

Model has been saved.
